In [ ]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:12 https://cloud.r-project.org/b

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2020-11-07 18:23:02--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.53MB/s    in 0.2s    

2020-11-07 18:23:02 (5.53 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Ebook_Purchase_v1_01.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   33605939| RGYFDX8QXKEIR|B007KO2MLO|     328837464|           Big Maria|Digital_Ebook_Pur...|          4|            0|          0|   N|                N|              Quirky|Elmore Leonard me...| 2013-09-09|
|         US|   34058393|R13CBGTMNV9R8Z|B005FLODDE|     764276359|The Woman Who Was...|Digital_Ebook_Pur

### Create DataFrames to match tables

In [ ]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame


In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RGYFDX8QXKEIR|          4|            0|          0|   N|                N|
|R13CBGTMNV9R8Z|          4|            1|          2|   N|                Y|
| R7DRFHC0F71O0|          5|            0|          0|   N|                N|
|R27LUKEXU3KBXQ|          5|            1|          1|   N|                Y|
|R1VXTPUYMNU687|          5|            1|          2|   N|                N|
|R30DKW1GJWLPZC|          3|            1|          2|   N|                Y|
|R18DPFG2FALJI9|          5|            0|          0|   N|                Y|
|R24D677N5WBW5Q|          5|            0|          0|   N|                Y|
|R2FCJ9BQLSIOR3|          5|            0|          0|   N|                Y|
|R1R6K4MAKDWTXI|          4|            0|          0|   N|     

In [ ]:
# Filter df where total_votes count is equal to or greater than 20
reviews_df = vine_df.filter(vine_df['total_votes'] >= 20)
reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R39ESX43X1SA5T|          1|           36|         47|   N|                Y|
|R31QRZ3JAMPBB4|          1|           16|         26|   N|                Y|
|R23FKBEXURC3SN|          5|           28|         36|   N|                N|
| R4IAIDV5EE84W|          1|            8|         21|   N|                Y|
|R26MB2DA7ROT4P|          1|            2|         21|   N|                Y|
| RTRVYV0GWUTCK|          5|           18|         25|   N|                Y|
|R3QLZAI37SNOOC|          2|           30|         31|   N|                Y|
|R1QYZU3RS4CY4F|          5|           66|         76|   N|                N|
|R2UR5OLN1BU87J|          2|           21|         23|   N|                Y|
| RYBZ6CTBOGI0H|          1|            3|         84|   N|     

In [ ]:
# Dataframe containing helpful votes more than 50%
helpful_df = reviews_df.filter(reviews_df['helpful_votes']/reviews_df['total_votes'] >= 0.5)
helpful_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R39ESX43X1SA5T|          1|           36|         47|   N|                Y|
|R31QRZ3JAMPBB4|          1|           16|         26|   N|                Y|
|R23FKBEXURC3SN|          5|           28|         36|   N|                N|
| RTRVYV0GWUTCK|          5|           18|         25|   N|                Y|
|R3QLZAI37SNOOC|          2|           30|         31|   N|                Y|
|R1QYZU3RS4CY4F|          5|           66|         76|   N|                N|
|R2UR5OLN1BU87J|          2|           21|         23|   N|                Y|
|R38NW44NY19VAZ|          5|           30|         32|   N|                Y|
| RLUFKZ223OR3L|          2|           20|         23|   N|                N|
|R3H9D0U7BCE6FU|          4|           23|         24|   N|     

In [ ]:
# Filter Dataframe where vine = Y
vinereview_df = helpful_df.filter(helpful_df['vine'] == 'Y')
vinereview_df.show()

+---------+-----------+-------------+-----------+----+-----------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+---------+-----------+-------------+-----------+----+-----------------+
+---------+-----------+-------------+-----------+----+-----------------+



In [ ]:
# Filter Dataframe where vine = N
nonpaid_df = helpful_df.filter(helpful_df['vine'] == 'N')
nonpaid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R39ESX43X1SA5T|          1|           36|         47|   N|                Y|
|R31QRZ3JAMPBB4|          1|           16|         26|   N|                Y|
|R23FKBEXURC3SN|          5|           28|         36|   N|                N|
| RTRVYV0GWUTCK|          5|           18|         25|   N|                Y|
|R3QLZAI37SNOOC|          2|           30|         31|   N|                Y|
|R1QYZU3RS4CY4F|          5|           66|         76|   N|                N|
|R2UR5OLN1BU87J|          2|           21|         23|   N|                Y|
|R38NW44NY19VAZ|          5|           30|         32|   N|                Y|
| RLUFKZ223OR3L|          2|           20|         23|   N|                N|
|R3H9D0U7BCE6FU|          4|           23|         24|   N|     

In [ ]:
total_vine = vinereview_df.count()
total_not = nonpaid_df.count()
print("The total number of paid reviews is %f" % total_vine)
print("The total number of unpaid reviews is %f" % total_not)

The total number of paid reviews is 0.000000
The total number of unpaid reviews is 65149.000000


In [ ]:
five_star_vine = vinereview_df.filter(vinereview_df['star_rating'] == 5)
five_star_vine_count = five_star_vine.count()
print("The total number of paid 5-star reviews is %f" % five_star_vine_count)
five_star_not = nonpaid_df.filter(nonpaid_df['star_rating'] == 5)
five_star_not_count = five_star_not.count()
print("The total number of unpaid 5-star reviews is %f" % five_star_not_count)

The total number of paid 5-star reviews is 0.000000
The total number of unpaid 5-star reviews is 24673.000000


In [ ]:
vine_percent_five = five_star_vine_count/total_vine * 100
not_percent_five = five_star_not_count/total_not * 100
print("For paid reviews, the percentage of 5-star reviews is %f" % vine_percent_five)
print("For unpaid reviews, the percentage of 5-star reviews is %f" % not_percent_five)

ZeroDivisionError: ignored